In [1]:
# Création du sommaire :
from jyquickhelper import add_notebook_menu
add_notebook_menu(first_level=1, last_level=4, header="SOMMAIRE")

In [16]:
import sklearn
sklearn.__version__

'0.22.1'

# Read Me

Reprendre la définition du projet et des objectifs

Détailler les instructions d'utilisation du notebook et de l'application Flask

Préciser les hypothèses

# Libraries

In [2]:
# Basic Librairies :
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import datetime as dt
import json

# Calcul de distance :
from geopy.distance import geodesic

# Pickle :
import pickle

# Functions & Hypothesis

In [3]:
# Functions :
def filter_df(df_to_filter, start_date, end_date) :
    """
    Function to reduce a dataframe on start/end date. 
    start_date and end_date must be in the form : YYYY-MM-DD hh:mm
    """ 
    start = dt.datetime.strptime(start_date, "%Y-%m-%d %H:%M")
    end = dt.datetime.strptime(end_date, "%Y-%m-%d %H:%M")
    return df_to_filter.loc[(df_to_filter["Datetime"] > start) & (df_to_filter["Datetime"] < end)]


def calcul_CO2(distance,label):
    """
    Function to calculate the CO2 emission in terms of distance and transportation means
    """
    return distance/1000*carbone_dict[label]


# Hypothesis on emissions by transport's mean (gCO2/km) [*source : ADEME (Hors infrastructure)] :
carbone_dict = {'AVION': 500,
                'ON_BICYCLE': 0,
                'ON_FOOT': 0,
                'ON_VEHICLE' : 220,
                'STILL': 0,
                'TGV': 1.9,
                'TRANSPORT_EN_COMMUN': 15}                              

# Datas' Pretreatment

In [4]:
# On charge notre dataset json dans un fichier raw :
datafile_path="Data_ED_2019-01-02/Historique des positions/Historique des positions.json"
with open(datafile_path, 'r') as fh:
    raw = json.loads(fh.read())
    
# On définit notre DataFrame, et on supprime le raw qui prend de l'espace mémoire :
df_location = pd.DataFrame(raw['locations'])
del raw

In [5]:
# conversion dans les unités internationales
df_location['latitudeE7'] = df_location['latitudeE7']/float(1e7) 
df_location['longitudeE7'] = df_location['longitudeE7']/float(1e7)
df_location['timestampMs'] = df_location['timestampMs'].map(lambda x: float(x)/1000) #(conversion en secondes)
df_location['Datetime'] = df_location.timestampMs.map(dt.datetime.fromtimestamp)

# On renomme nos colonnes :
df_location.rename(columns={'latitudeE7':'Latitude', 'longitudeE7':'Longitude', 'timestampMs':'Timestamp'}, inplace=True)

In [6]:
# Réduction éventuelle de la taille du dataset, par date :
start_date = "2019-12-01 00:00"
end_date = "2019-12-31 00:00"

df_location = filter_df(df_location, start_date, end_date) 

In [7]:
# On supprime les lignes dont la précision de la position est < 500m :
min_precision = 500     # en mètre
df_location = df_location[abs(df_location.accuracy) < min_precision]

In [8]:
# Création d'un nouveau dataset :
df = df_location.reset_index(drop=True).copy(deep=True)
df_path = df.copy(deep=True)

In [9]:
# Définition des colonnes dont j'ai besoin dans mon dataset Trajet :
columns = ['Dep_Datetime',
           'Dep_Time', 'Arr_Time', 
           'Dep_Lat', 'Arr_Lat', 'Dep_Lon', 'Arr_Lon',
           'Duration', 'Distance', 'Velocity'
          ]

In [10]:
# Création des colonnes dans mon nouveau dataset :

# Date des positions dep
df_path['Dep_Datetime'] = df['Datetime']

# Timestamp des positions dep/arr
df_path['Dep_Time'] = df['Timestamp']
df_path['Arr_Time'] = df.shift(periods=-1)['Timestamp']

# Latitude des positions dep/arr
df_path['Dep_Lat'] = df['Latitude']
df_path['Arr_Lat'] = df.shift(periods=-1)['Latitude']

# Longitude des positions dep/arr
df_path['Dep_Lon'] = df['Longitude']
df_path['Arr_Lon'] = df.shift(periods=-1)['Longitude']

# Je supprime la dernière ligne qui ne représente pas de trajet et contient des NaN :
df_path = df_path[:-1]

# Colonne 'Duration' (duréee en secondes)
df_path['Duration'] = df_path['Arr_Time'] - df_path['Dep_Time']

# Colonne 'Distance' (longueur du déplacement en mètre) :
df_path['Distance'] = [geodesic((a, b),(c, d)).m for (a,b,c,d) 
                         in zip(df_path.Dep_Lat, df_path.Dep_Lon, df_path.Arr_Lat, df_path.Arr_Lon)]

# Colonne 'Velocity' (Vitesse de déplacement en km/h)
df_path['Velocity'] = ( df_path['Distance'] / 1000 ) / ( df_path['Duration'] / 3600 )

# Je supprime les colonnes inutiles :
df_path = df_path[columns]

# Je supprime les lignes dont la durée du trajet est inférieure à 5 secondes :
df_path = df_path[df_path['Duration'] >= 5]

In [11]:
# Copie du dataset en fin de traitement :
df = df_path.reset_index(drop=True).copy(deep=True)

# ML Model predictions

In [12]:
# Choix des featurees :
features_list = ['Dep_Time', 'Arr_Time', 
                 'Dep_Lat', 'Arr_Lat', 'Dep_Lon', 'Arr_Lon', 
                 'Duration', 'Distance', 'Velocity']

X = df.loc[:,features_list]

In [13]:
# loading the model from disk :
filename = 'model_rf.sav'
model = pickle.load(open(filename, 'rb'))

# Prediction on dataset :
y_pred = model.predict(X)

# Fusion des labels prédits avec les features :
df_pred = X
df_pred['Label'] = y_pred

# attribution des labels :
label_filename = "./Pickle_Saved/label_dictionary.pickle"
with open(label_filename, 'rb') as handle:
    labels = pickle.load(handle)
    
df_pred['Label'] = df_pred['Label'].map(labels)

# Bilan CO2

In [14]:
# Calcul de l'impact carbone par micro trajet :
df_pred["Carbon_Impact"] = [calcul_CO2(x, y) for (x, y) in zip(df_pred["Distance"], df_pred["Label"])]

# Calcul de l'impact carbone total sur la période de temps choisit :
Total_Impact_Kg = df_pred["Carbon_Impact"].sum()/1000

print("Votre impact carbone sur la période allant du {} au {} est de : {:.2f} kg  ".format(start_date, end_date, Total_Impact_Kg))
print("Envisagez une compensation de {:.2f} € TTC, par exemple via GoodPlanet ".format(Total_Impact_Kg/1000*22))

Votre impact carbone sur la période allant du 2019-12-01 00:00 au 2019-12-31 00:00 est de : 3386.61 kg  
Envisagez une compensation de 74.51 € TTC, par exemple via GoodPlanet 


In [18]:
# Saving results :
name = 'Carbon_results_ED_01'

df_pred.to_csv("./Pickle_Saved/{}.csv".format(name))
print("Les résultats de votre bilan carbon ont été sauvegardés au format *.csv dans './Pickle_Saved/{}.csv'".format(name))

Les résultats de votre bilan carbon ont été sauvegardés au format *.csv dans './Pickle_Saved/Carbon_results_ED_01.csv'
